In [91]:
import pandas as pd
import numpy as np

In [92]:
clicks = pd.read_csv("/home/ailen-magali/Documentos/clicks.csv.gzip")

In [93]:
#Me quedo con las columnas que me importan
clicks = clicks.drop(columns=['agent_device', 'os_minor', 'os_major', 'country_code', 'action_id',\
                'brand', 'touchX', 'touchY', 'specs_brand', 'carrier_id', 'longitude', 'latitude', 'trans_id'], axis=1)

Creando Features

In [94]:
#Muestro las columnas con las que me quedo
clicks.head()

,advertiser_id,source_id,created,wifi_connection,timeToClick,ref_type,ref_hash
0,1,2,2019-04-18T05:27:42.197Z,False,2.317,1891515180541284343,1293710398598742392
1,1,1,2019-04-18T05:27:03.164Z,True,7.653,1891515180541284343,1663930990551616564
2,1,1,2019-04-18T05:42:07.926Z,True,464.796,1891515180541284343,8488038938665586188
3,1,1,2019-04-18T05:26:04.446Z,True,225.311,1891515180541284343,6488361690105189959
4,1,1,2019-04-18T05:23:37.764Z,True,84.736,1891515180541284343,1348993302102753419


In [95]:
clicks['wifi_connection'] = clicks['wifi_connection'].map({True: 1, False: 0})
clicks['wifi_connection'] = clicks['wifi_connection'].fillna(-1)

In [96]:
clicks['created'] = pd.to_datetime(clicks['created'], format='%Y-%m-%d %H:%M:%S')

In [97]:
clicks['ref_type_android'] = (clicks['ref_type'].astype(str).str.contains("1891515180541284343") == True).astype(int)
clicks['ref_type_iphone'] = (clicks['ref_type'].astype(str).str.contains("1494519392962156891") == True).astype(int)

clicks = clicks.drop('ref_type', axis=1)

In [98]:
clicks['advertiser_id'].value_counts()

2    36135
1    27433
0      516
3      211
4        1
Name: advertiser_id, dtype: int64

In [99]:
clicks['advertiser_0'] = (clicks['advertiser_id'].astype(str).str.contains("0") == True).astype(int)
clicks['advertiser_1'] = (clicks['advertiser_id'].astype(str).str.contains("1") == True).astype(int)
clicks['advertiser_2'] = (clicks['advertiser_id'].astype(str).str.contains("2") == True).astype(int)
clicks['advertiser_3'] = (clicks['advertiser_id'].astype(str).str.contains("3") == True).astype(int)
clicks['advertiser_4'] = (clicks['advertiser_id'].astype(str).str.contains("4") == True).astype(int)

clicks = clicks.drop('advertiser_id', axis=1)

In [100]:
clicks['source_id'].value_counts()

1    33695
0    17155
3     5639
2     3878
6     2186
4      823
5      454
7      329
8      137
Name: source_id, dtype: int64

In [101]:
clicks['source_0'] = (clicks['source_id'].astype(str).str.contains("0") == True).astype(int)
clicks['source_1'] = (clicks['source_id'].astype(str).str.contains("1") == True).astype(int)
clicks['source_2'] = (clicks['source_id'].astype(str).str.contains("2") == True).astype(int)
clicks['source_3'] = (clicks['source_id'].astype(str).str.contains("3") == True).astype(int)
clicks['source_4'] = (clicks['source_id'].astype(str).str.contains("4") == True).astype(int)
clicks['source_5'] = (clicks['source_id'].astype(str).str.contains("5") == True).astype(int)
clicks['source_6'] = (clicks['source_id'].astype(str).str.contains("6") == True).astype(int)
clicks['source_7'] = (clicks['source_id'].astype(str).str.contains("7") == True).astype(int)
clicks['source_8'] = (clicks['source_id'].astype(str).str.contains("8") == True).astype(int)

clicks = clicks.drop('source_id', axis=1)

In [102]:
#clicks['feriado'] = (clicks['created'].dt.day == 19).map({True: 1, False: 0})
#clicks['feriado'] = clicks['feriado'].fillna(-1)

In [103]:
clicks['weekday'] = clicks['created'].dt.weekday

clicks = pd.get_dummies(clicks, columns=['weekday'])

In [105]:
clicks = clicks.rename(index=str, columns={"weekday_0": "lunes", "weekday_1": "martes", "weekday_2": "miercoles", \
                                           "weekday_3": "jueves", "weekday_4": "viernes", "weekday_5": "sabado", \
                                           "weekday_6": "domingo"})

In [106]:
bins = [0, 6, 13, 17, 25]
labels = ['morning','afternoon','evening','night']
hours = clicks['created'].dt.hour
clicks['time'] = pd.cut(hours-5+24 *(hours<5),bins=bins,labels=labels,right=False)

clicks = pd.get_dummies(clicks, columns=['time'])

In [113]:
clicks['feriado'] = (clicks['created'].dt.day).map({19: 1})
clicks['feriado'] = clicks['feriado'].fillna(0)

In [115]:
clicks.head().T

,0,1,2,3,4
created,2019-04-18 05:27:42.197000+00:00,2019-04-18 05:27:03.164000+00:00,2019-04-18 05:42:07.926000+00:00,2019-04-18 05:26:04.446000+00:00,2019-04-18 05:23:37.764000+00:00
wifi_connection,0,1,1,1,1
timeToClick,2.317,7.653,464.796,225.311,84.736
ref_hash,1293710398598742392,1663930990551616564,8488038938665586188,6488361690105189959,1348993302102753419
ref_type_android,1,1,1,1,1
ref_type_iphone,0,0,0,0,0
advertiser_0,0,0,0,0,0
advertiser_1,1,1,1,1,1
advertiser_2,0,0,0,0,0
advertiser_3,0,0,0,0,0
